# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Jun-2025 11:19:40


## 1. Grounding Bing Search

### Connect to AI Foundry

In [5]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [6]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing
bing.definitions


Created agent, ID: asst_LbPiTMVBKkkQCBLJihzFiMap


[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbinggrounding002', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [8]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_CARhdVJPrBxeSqLu2wi2dmHg


### Example #1

In [9]:
query = "What is the current price of MICROSOFT stock?"

### Create first message in the thread

In [10]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_HprqR2x32HvR7GNYtw23NUKW


### Create first RUN

In [11]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [12]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_k9HE7TXyEWveN4Ont5qVXFae status: completed
Step step_gYbYK6rhnWIAypTTUw3pa61A status: completed
  Tool calls:
    Tool Call ID: call_TZQBgiLoxIJvZIBLm7zUz9GA
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current price of Microsoft stock")



{'prompt_tokens': 2654, 'completion_tokens': 91, 'total_tokens': 2745, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [13]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The current price of Microsoft (MSFT) stock is $474.96 as of the latest available market data. After-hours trading shows it slightly lower at $474.21. Please note the market is closed and prices may change when it opens again【3:4†Microsoft Stock Price Today | NASDAQ: MSFT Live - Investing.com】.
URL Citation: [Microsoft Stock Price Today | NASDAQ: MSFT Live - Investing.com](https://www.investing.com/equities/microsoft-corp)


### Example #2

In [14]:
query = "find out about the latest iPhone"

In [15]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_PO82sbt4eEFNWRvwDEoCtyRV


In [16]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [17]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_iRL5m9yT0v1FwG1PSnlOyiO4 status: completed
Step step_wwlLsPYoEbnC4qxilY8uSkso status: completed
  Tool calls:
    Tool Call ID: call_JOlSRjQsbww4VRnfGTrPlNB9
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "latest iPhone release 2025")



{'prompt_tokens': 5207, 'completion_tokens': 301, 'total_tokens': 5508, 'prompt_token_details': {'cached_tokens': 0}}

In [18]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone is the upcoming iPhone 17 series, set to launch in September 2025. Notable changes include a significant design overhaul and the introduction of a brand-new, ultra-slim model called the "iPhone 17 Air." There are expected to be four main models: iPhone 17, iPhone 17 Pro, iPhone 17 Pro Max, and iPhone 17 Air. The "Plus" model is reportedly being replaced by the Air.

Key updates and rumors about the iPhone 17 series include:
- ProMotion (120Hz) displays for all models, not just the Pro line.
- An all-new design for the Pro models, featuring larger screens and updated camera arrays.
- The new iPhone 17 Air is expected to be extremely thin, with a display around 6.6 inches.
- Camera improvements, especially for selfies, with a bump up to a 24MP front camera.
- The iPhone SE 4 will adopt a modern, all-screen design and upgraded internals.
- All models may feature Apple’s in-house 5G modem and enhanced AI features via iOS 19 or later updates【7:3†MacRumors】【

### Example #3

In [19]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [20]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_lj0oUkL9T6K9qcv44YuG2Fpc


In [21]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [22]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_4eX9OevUBA7BmtON9cpJGMZ6 status: completed
Step step_rXE7HdwNQUXVoJhKuRVeSLEk status: completed
  Tool calls:
    Tool Call ID: call_M1XYEuBWJvqPWUwdm1aYqSjn
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "Mexico City weather today")



{'prompt_tokens': 5720, 'completion_tokens': 279, 'total_tokens': 5999, 'prompt_token_details': {'cached_tokens': 0}}

In [23]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: Here's your funny Mexico City weather report! 🌧️🌩️☁️

Right now, Mexico City is living its best dramatic telenovela life—there’s rain in the forecast and thunder is likely to make a cameo! The temperature is a cozy 21°C (about 70°F). Humidity is at 62%—so if your hair looks extra puffy, now you know why! 💁

You’ll want an umbrella unless you’re auditioning for the next aquatic Olympics—rain is expected for several hours. If you step outside and hear a rumble, don’t worry; it’s just the sky yelling about the tacos it can’t have. Expect max temps around 22°C (72°F) and minimums at 13°C (55°F).

Bonus: Air quality is generally OK, but if you’re sensitive, channel your inner masked luchador! 🤼‍♂️

So, wear your rain boots, grab that umbrella, and maybe practice your most dramatic slow-motion run through puddles. 🌧️💨🌈

Stay dry—unless you’re a plant! 🌱

Sources:
- current temps, rain, and forecast details【11:0†Weather.com】【11:2†Weather-atlas.com】
URL Citation: [Weather Forec

### Post processing

In [24]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_LbPiTMVBKkkQCBLJihzFiMap, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing


In [26]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [27]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")